In [2]:
:dep ndarray = "^0.15.0"
:dep ndarray-stats = "0.5.1"
:dep mnist = { version = "0.5.0", features = ["download"] }
:dep plotters = { version = "^0.3.4", default_features = false, features = ["evcxr", "all_series"] }
:dep image = "0.23"
:dep evcxr_image = "1.1.0"

use ndarray::{s, array, Array, Array1, Array2, ArrayBase, Ix1, Ix2, Ix3, Zip};
use ndarray_stats::{QuantileExt};
use mnist::*;
extern crate plotters;
use plotters::prelude::*;
use image::{ImageBuffer, RgbImage, Rgb};
use evcxr_image::ImageDisplay;

## Perceptron Logic gate

$$y = \begin{cases}0 \quad (w_1x_1+w_2x_2\le\theta)\\1 \quad (w_1x_2+w_2x_2>\theta)\end{cases}$$

In [3]:
fn simple_AND(x1: f32, x2: f32) -> f32 {
    let w1 = 0.5;
    let w2 = 0.5;
    let theta = 0.7_f32;
    let temp = x1 * w1 + x2 * w2;
    if temp <= theta {
        0.0_f32
    } else {
        1.0_f32
    }
}

In [4]:
assert!(simple_AND(0., 0.) == 0.);
assert!(simple_AND(1., 0.) == 0.);
assert!(simple_AND(0., 1.) == 0.);
assert!(simple_AND(1., 1.) == 1.);

$$y = \begin{cases}0 \quad (b+w_1x_1+w_2x_2\le0)\\1 \quad (b+w_1x_2+w_2x_2>0)\end{cases}$$

$$y = h(b+w_1x_1+w_2x_2)$$

$$h(x) = \begin{cases}0 \quad (x\le0)\\1 \quad (x>0)\end{cases}$$

In [5]:
fn AND(x1: f32, x2: f32) -> f32 {
    let x = array![x1, x2];
    let weight = array![0.5_f32, 0.5];
    let bias = -0.7_f32;
    let temp = x.dot(&weight) + bias;
    if temp <= 0. {
        0.0_f32
    } else {
        1.0_f32
    }
}

In [6]:
assert!(AND(0., 0.) == 0.);
assert!(AND(1., 0.) == 0.);
assert!(AND(0., 1.) == 0.);
assert!(AND(1., 1.) == 1.);

In [7]:
fn NAND(x1: f32, x2: f32) -> f32 {
    let x = array![x1, x2];
    let weight = array![-0.5_f32, -0.5];
    let bias = 0.7_f32;
    let temp = x.dot(&weight) + bias;
    if temp <= 0. {
        0.0_f32
    } else {
        1.0_f32
    }
}

In [8]:
assert!(NAND(0., 0.) == 1.);
assert!(NAND(1., 0.) == 1.);
assert!(NAND(0., 1.) == 1.);
assert!(NAND(1., 1.) == 0.);

In [9]:
fn OR(x1: f32, x2: f32) -> f32 {
    let x = array![x1, x2];
    let weight = array![0.5_f32, 0.5];
    let bias = -0.2_f32;
    let temp = x.dot(&weight) + bias;
    if temp <= 0. {
        0.0_f32
    } else {
        1.0_f32
    }
}

In [10]:
assert!(OR(0., 0.) == 0.);
assert!(OR(1., 0.) == 1.);
assert!(OR(0., 1.) == 1.);
assert!(OR(1., 1.) == 1.);

In [11]:
fn XOR(x1: f32, x2: f32) -> f32 {
    let s1 = NAND(x1, x2);
    let s2 = OR(x1, x2);
    AND(s1, s2)
}

In [12]:
assert!(XOR(0., 0.) == 0.);
assert!(XOR(1., 0.) == 1.);
assert!(XOR(0., 1.) == 1.);
assert!(XOR(1., 1.) == 0.);

## Activation Function

$$a=b+x_1x_1+w_2x_2$$

$$y=h(a)$$

In [13]:
fn draw_figure(mut x_data: Array1<f32>, y_data: Array1<f32>, xlim: (f32, f32), ylim: (f32, f32)) {
    evcxr_figure((480, 320), |root| {
        root.fill(&WHITE)?;
        let mut chart = ChartBuilder::on(&root)
        .margin(5)
        .x_label_area_size(30)
        .y_label_area_size(30)
        .build_cartesian_2d(xlim.0..xlim.1, ylim.0..ylim.1)?;

        chart.configure_mesh().disable_mesh().draw()?;

        chart.draw_series(LineSeries::new(
            Zip::from(&mut x_data).and(&y_data).map_collect(|x, y| (*x, *y)),
            &BLACK
        ));
        Ok(())
    }).evcxr_display()
}

In [14]:
fn simple_step(x: f32) -> i32 {
    if x > 0. {1} else {0}
}

assert!(simple_step(0.1) == 1);
assert!(simple_step(-0.1) == 0);
assert!(simple_step(0.5) == 1);
assert!(simple_step(2.3) == 1);

fn step_array(arr_x: &Array1<f32>) -> Array1<f32> {
    arr_x.map(|x| if *x > 0. {1.} else {0.})
}

assert!(step_array(&array![-1.0_f32, 1.0, 2.0]) == array![0.0_f32, 1.0, 1.0]);
let x = Array::range(-5.0_f32, 5.0, 0.1);
let y = step_array(&x);
draw_figure(x, y, (-6.0, 6.0), (-0.1, 1.1))

()

In [15]:
fn sigmoid_array(arr_x: &Array1<f32>) -> Array1<f32> {
    arr_x.map(|x| 1. / (1. + (-x).exp()))
}

assert!(sigmoid_array(&array![-1.0, 1.0, 2.0]) == array![0.26894143, 0.7310586, 0.880797]);

let x = Array::range(-5.0_f32, 5.0, 0.1);
let y = sigmoid_array(&x);
draw_figure(x, y, (-6.0, 6.0), (-0.1, 1.1))

()

In [16]:
fn relu(arr_x: &Array1<f32>) -> Array1<f32> {
    arr_x.map(|x| x.max(0.))
}

assert!(relu(&array![-1.0, 0.5, 2.0]) == array![0., 0.5, 2.0]);

let x = Array::range(-5.0_f32, 5.0, 0.1);
let y = relu(&x);
draw_figure(x, y, (-6.0, 6.0), (-1.1, 5.1))

()

## Multi-dimensional array calculation

In [17]:
let A = array![1, 2, 3, 4];
println!("{}", &A);
println!("{}", &A.ndim());
println!("{:?}", &A.shape());
println!("{}", &A.shape().get(0).unwrap());
println!("{}", &A.dim());

[1, 2, 3, 4]
1


In [18]:
let B = array![[1, 2], [3, 4], [5, 6]];
println!("{}", &B);
println!("{}", &B.ndim());
println!("{:?}", &B.dim());
println!("{:?}", &B.shape());

[[1, 2],


In [19]:
let A = array![[1, 2], [3, 4]];
println!("{:?}", &A.shape());
let B = array![[5, 6], [7, 8]];
println!("{:?}", &B.shape());
println!("{}", &A.dot(&B));

In [20]:
let A = array![[1, 2, 3], [4, 5, 6]];
println!("{:?}", &A.shape());
let B = array![[1, 2], [3, 4], [5, 6]];
println!("{:?}", &B.shape());
println!("{}", &A.dot(&B));

[2, 3]


In [21]:
let C = array![[1, 2], [3, 4]];
println!("{:?}", &C.shape());
println!("{:?}", &A.shape());

In [22]:
std::panic::catch_unwind(|| {
    &A.dot(&C);
});

thread '<unnamed>' panicked at 'ndarray: inputs 2 × 3 and 2 × 2 are not compatible for matrix multiplication', /home/noxowl/.cargo/registry/src/github.com-1ecc6299db9ec823/ndarray-0.15.6/src/linalg/impl_linalg.rs:299:5
stack backtrace:
   0: rust_begin_unwind
             at /rustc/897e37553bba8b42751c67658967889d11ecd120/library/std/src/panicking.rs:584:5
   1: core::panicking::panic_fmt
             at /rustc/897e37553bba8b42751c67658967889d11ecd120/library/core/src/panicking.rs:142:14
   2: ndarray::linalg::impl_linalg::dot_shape_error
   3: ndarray::linalg::impl_linalg::<impl ndarray::ArrayBase<S,ndarray::dimension::dim::Dim<[usize; 2]>>>::dot
   4: <core::panic::unwind_safe::AssertUnwindSafe<F> as core::ops::function::FnOnce<()>>::call_once
   5: run_user_code_21
   6: evcxr::runtime::Runtime::run_loop
   7: evcxr::runtime::runtime_hook
   8: evcxr_jupyter::main


In [23]:
let A = array![[1, 2], [3, 4], [5, 6]];
println!("{:?}", &A.shape());
let B = array![7, 8];
println!("{:?}", &B.shape());
println!("{}", &A.dot(&B));

[3, 2]


## Neural network dot product

$$ X(2) W(2*3) = Y(3)$$

In [24]:
let X = array![1, 2];
println!("{:?}", &X.shape());
let W = array![[1, 3, 5], [2, 4, 6]];
println!("{}", &W);
println!("{:?}", &W.shape());
let Y = X.dot(&W);
println!("{}", &Y);

[2]


### Input layer to Hidden layer 1

In [25]:
let X = array![1.0_f32, 0.5];
let W1 = array![[0.1_f32, 0.3, 0.5], [0.2, 0.4, 0.6]];
let B1 = array![0.1_f32, 0.2, 0.3];

println!("{:?}", &W1.shape());
println!("{:?}", &X.shape());
println!("{:?}", &B1.shape());

let A1 = X.dot(&W1) + &B1;
let Z1 = sigmoid_array(&A1);
println!("{}", &A1);
println!("{}", &Z1);

[2, 3]
[2]


### Hidden layer 1 to layer 2

In [26]:
let W2 = array![[0.1_f32, 0.4], [0.2, 0.5], [0.3, 0.6]];
let B2 = array![0.1_f32, 0.2];

println!("{:?}", &Z1.shape());
println!("{:?}", &W2.shape());
println!("{:?}", &B2.shape());

let A2 = Z1.dot(&W2) + &B2;
let Z2 = sigmoid_array(&A2);
println!("{}", &A2);
println!("{}", &Z2);

[3]
[3, 2]


### Hidden layer 2 to output layer

In [27]:
fn identify(x: Array1<f32>) -> Array1<f32> { x }

let W3 = array![[0.1_f32, 0.3], [0.2, 0.4]];
let B3 = array![0.1_f32, 0.2];

let A3 = Z2.dot(&W3) + &B3;
let Y = identify(A3);
println!("{}", &Y);

[0.3168271, 0.6962791]


### Identify function and Softmax function

$$y_k = {\exp(a_k) \over \sum_{i=1}^n \exp(a_i)}$$

In [28]:
let a = array![0.3_f32, 2.9, 4.0];
let exp_a = a.map(|x| x.exp());
println!("{}", &exp_a);
let sum_exp_a = exp_a.sum();
println!("{}", &sum_exp_a);
let y = exp_a / sum_exp_a;
println!("{}", &y);

fn softmax(a: &Array1<f32>) -> Array1<f32> {
    let exp_a = a.map(|x| x.exp());
    let sum_exp_a = exp_a.sum();
    exp_a / sum_exp_a
}

assert!(y == softmax(&array![0.3, 2.9, 4.0]));

[1.3498589, 18.174147, 54.59815]
74.122154
[0.018211273, 0.24519183, 0.7365969]


In [29]:
let a = array![1010.0_f32, 1000., 990.];
let exp_a = a.map(|x| x.exp());
println!("{}", &exp_a);
let sum_exp_a = exp_a.sum();
println!("{}", &sum_exp_a);
let y = exp_a / sum_exp_a;
println!("{}", &y);


[inf, inf, inf]


In [30]:
let a = array![1010.0_f32, 1000., 990.];
let c: f32 = *a.max()?;
let a_mod = a - c;
println!("{}", &a_mod);
let exp_a = a_mod.map(|x| x.exp());
println!("{}", &exp_a);
let sum_exp_a = exp_a.sum();
println!("{}", &sum_exp_a);
let y = exp_a / sum_exp_a;
println!("{}", &y);

fn softmax(a: &Array1<f32>) -> Array1<f32> {
    let c: f32 = *a.max().unwrap();
    let a_mod = a - c;
    let exp_a = a_mod.map(|x| x.exp());
    let sum_exp_a = exp_a.sum();
    exp_a / sum_exp_a
}

[0, -10, -20]
[1, 0.00004539993, 0.0000000020611537]
1.0000454
[0.9999546, 0.00004539787, 0.00000000206106]


In [31]:
let a = array![0.3, 2.9, 4.0];
let y = softmax(&a);
println!("{}", &y);
println!("{:?}", y.sum());

[0.018211273, 0.24519183, 0.73659694]


## Recognize handwritten digits by forward propagation method (MNIST Database)

In [32]:
let Mnist {
    trn_img, trn_lbl,
    tst_img, tst_lbl,
    ..
} = MnistBuilder::new()
    .label_format_digit()
    .training_set_length(60_000)
    .test_set_length(10_000)
    .download_and_extract()
    .finalize();

Attempting to download and extract train-images-idx3-ubyte.gz...
  File "data/train-images-idx3-ubyte.gz" already exists, skipping downloading.
  File "data/train-labels-idx1-ubyte.gz" already exists, skipping downloading.
  File "data/t10k-images-idx3-ubyte.gz" already exists, skipping downloading.
  File "data/t10k-labels-idx1-ubyte.gz" already exists, skipping downloading.
  Extracted file "data/train-images-idx3-ubyte" already exists, skipping extraction.
Attempting to download and extract train-labels-idx1-ubyte.gz...
  File "data/train-images-idx3-ubyte.gz" already exists, skipping downloading.
  File "data/train-labels-idx1-ubyte.gz" already exists, skipping downloading.
  File "data/t10k-images-idx3-ubyte.gz" already exists, skipping downloading.
  File "data/t10k-labels-idx1-ubyte.gz" already exists, skipping downloading.
  Extracted file "data/train-labels-idx1-ubyte" already exists, skipping extraction.
Attempting to download and extract t10k-images-idx3-ubyte.gz...
  File "

In [33]:
let x_train = Array2::from_shape_vec((60_000, 784), trn_img)
    .expect("Error converting images to Array2 struct")
    .map(|x| *x as f32);
let t_train = Array2::from_shape_vec((60_000, 1), trn_lbl)
    .expect("Error converting images to Array2 struct")
    .map(|x| *x as i32);
let x_test = Array2::from_shape_vec((10_000, 784), tst_img)
    .expect("Error converting images to Array2 struct")
    .map(|x| *x as f32);
let t_test = Array2::from_shape_vec((10_000, 1), tst_lbl)
    .expect("Error converting images to Array2 struct")
    .map(|x| *x as i32);
println!("{:?}", &x_train.shape());
println!("{:?}", &t_train.shape());
println!("{:?}", &x_test.shape());
println!("{:?}", &t_test.shape());

[60000, 784]
[60000, 1]
[10000, 784]
[10000, 1]


In [34]:
fn imshow(flatten_img: &Array1<f32>, width: u32, height: u32) {
    let img_rgb = RgbImage::new(width, height);
    image::ImageBuffer::from_fn(width, height, | x, y | {
        let yi = y * width;
        let i = (x + yi) as usize;
        let p = flatten_img[i] as u8;
        image::Rgb([p, p, p])
    }).evcxr_display()
}

In [35]:
let img: Array1<f32> = x_train.slice(s![0, ..]).to_owned();
let label: Array1<i32> = t_train.slice(s![0, ..]).to_owned();
println!("{}", &label);
println!("{:?}", &img.shape());
imshow(&img, 28, 28)

[5]
[784]


In [37]:
fn get_data() -> (Array2<f32>, Array2<i32>) {
    let Mnist {
        trn_img, trn_lbl,
        tst_img, tst_lbl,
        ..
    } = MnistBuilder::new()
        .label_format_digit()
        .training_set_length(60_000)
        .test_set_length(10_000)
        .download_and_extract()
        .finalize();
    let x_test = Array2::from_shape_vec((10_000, 784), tst_img)
        .expect("Error converting images to Array2 struct")
        .map(|x| *x as f32);
    let t_test = Array2::from_shape_vec((10_000, 1), tst_lbl)
        .expect("Error converting images to Array2 struct")
        .map(|x| *x as i32);
    (x_test, t_test)
}